# Hospital Facilities Insight 

Data obtained from Community Benefits Insight: http://www.communitybenefitinsight.org/api/get_hospitals.php

## Import Libaries

In [189]:
# import appropriate packages
import os
import requests
import json 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql.cursors
from pymysql import IntegrityError

import warnings
warnings.filterwarnings("ignore") #supress warnings

## Import Data

In [190]:
# API endpoint
api_url = 'http://www.communitybenefitinsight.org/api/get_hospitals.php'

# requesting data from API
response = requests.get(api_url)

# check status
if response.status_code == 200:
    data = response.json()

    # data update check
    last_modified_header = response.headers.get('Last-Modified')

    if last_modified_header:
        last_modified = datetime.strptime(last_modified_header, '%a, %d %b %Y %H:%M:%S GMT')
        print(f"Data was last modified: {last_modified}")

    # convert data to dataframe in chunks
    chunk_size = 100  # define size of chunk
    hospital_dfs = []  # create list to store df 

    for chunk_start in range(0, len(data), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(data))
        chunk_data = data[chunk_start:chunk_end]
        hospital_df = pd.DataFrame(chunk_data)
        hospital_dfs.append(hospital_df)

    # concate all df chunks into one
    hospital_df = pd.concat(hospital_dfs, ignore_index=True)

    # print df 
    print(hospital_df)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

     hospital_id hospital_org_id        ein  \
0              1               1  630307951   
1              2               2  630578923   
2              3               3  630312913   
3              4               4  630459034   
4              5               5  581973570   
...          ...             ...        ...   
3486        3487            2647  813040663   
3487        3488            2304  741109643   
3488        3489            2648  831954982   
3489        3490            2302  750800661   
3490        3491            2649  831869297   

                                     name  \
0                Mizell Memorial Hospital   
1                        St Vincents East   
2           Shelby Baptist Medical Center   
3            Callahan Eye Foundation Hosp   
4                 Cherokee Medical Center   
...                                   ...   
3486          Bsw Medical Center - Austin   
3487              Ascension Seton Bastrop   
3488         Texas Health Hosp

In [191]:
hospital_df # print df 

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,01039,99,N,N,N,N,010007,Covington County,<100 beds,"November 20, 2023"
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,01073,362,N,Y,N,Y,010011,Jefferson County,>299 beds,"November 20, 2023"
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,01117,252,N,Y,N,N,010016,Shelby County,100-299 beds,"November 20, 2023"
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,01073,106,N,Y,N,Y,010018,Jefferson County,100-299 beds,"November 20, 2023"
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,01019,60,N,N,N,N,010022,Cherokee County,<100 beds,"November 20, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,16,N,Y,N,N,670136,Travis County,<100 beds,"November 20, 2023"
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,7,N,Y,N,N,670143,Bastrop County,<100 beds,"November 20, 2023"
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,63,N,Y,N,N,670260,Denton County,<100 beds,"November 20, 2023"
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,46,N,Y,N,N,670300,Ellis County,<100 beds,"November 20, 2023"


In [192]:
# print 10 random samples from the dataframe 
hospital_df.sample(10)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
2055,2056,1620,111704595,Long Island Community Hospital,Long Island Community Hospital,101 Hospital Road,E Patchogue,NY,11772,36103,306,N,Y,N,N,330141,Suffolk County,>299 beds,"November 20, 2023"
2399,2400,1906,341407259,Mary Rutan Hospital,Mary Rutan Hospital,205 Palmer Avenue,Bellefontaine,OH,43311,39091,115,N,N,N,N,360197,Logan County,100-299 beds,"November 20, 2023"
2658,2659,2101,250965598,Warren General Hospital,Warren General Hospital,2 Crescent Park West,Warren,PA,16365,42123,85,N,N,N,N,390146,Warren County,<100 beds,"November 20, 2023"
813,814,592,370663567,St Elizabeth Hospital,St Elizabeth Hospital,1 Saint Elizabeth Blvd,Ofallon,IL,62269,17163,260,Y,Y,N,Y,140187,St. Clair County,100-299 beds,"November 20, 2023"
155,156,111,954896822,Dewitt Hospital & Nursing Home Inc,Dewitt Hospital & Nursing Home Inc,Highway 1 & Madison,Dewitt,AR,72042,05001,25,N,N,N,N,041314,Arkansas County,<100 beds,"November 20, 2023"
1669,1670,1300,721379812,Jefferson Davis Community Hospital Cah,Prentiss Regional Hospital,1102 Rose Street,Prentiss,MS,39474,28065,35,N,N,N,N,251326,Jefferson Davis County,<100 beds,"November 20, 2023"
2052,2053,1617,452964467,Bon Secours Community Hospital,Bon Secours Community Hospital,160 East Main Street,Port Jervis,NY,12771,36071,141,Y,Y,N,N,330135,Orange County,100-299 beds,"November 20, 2023"
2609,2610,2062,230906220,Nason Hospital,,105 Nason Drive,Roaring Spring,PA,16673,42013,42,N,Y,N,N,390062,Blair County,<100 beds,"November 20, 2023"
2522,2523,1996,930223960,Three Rivers Medical Center,Three Rivers Medical Center,500 Sw Ramsey Ave,Grants Pass,OR,97527,41033,150,N,Y,N,N,380002,Josephine County,100-299 beds,"November 20, 2023"
477,478,304,060646741,Milford Hospital Inc,Milford Hospital Inc,300 Seaside Avenue,Milford,CT,06450,09009,106,N,Y,N,N,070019,New Haven County,100-299 beds,"November 20, 2023"


## Data Inspection

In [193]:
hospital_df.shape 

(3491, 19)

In [194]:
hospital_df.info() #all variables in df are labeled as object variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   object
 11  chrch_affl_f                3491 non-null   object
 12  urban_location_f            3491 non-null   object
 13  children_hospital_f         3491 non-null   obje

In [195]:
columns_to_convert = ['hospital_bed_count', 'medicare_provider_number']
binary_columns = ['chrch_affl_f', 'urban_location_f', 'children_hospital_f', 'memb_counc_teach_hosps_f']

# iterate through the list of columns
for col in columns_to_convert:
    # convert to numeric, coerce errors to NaN
    hospital_df[col] = pd.to_numeric(hospital_df[col], errors='coerce')
    
for col in binary_columns:
    # Map 'N' to 0 and 'Y' to 1
    hospital_df[col] = hospital_df[col].map({'N': 0, 'Y': 1})

# convert updated_dt to date type
hospital_df['updated_dt'] = hospital_df['updated_dt'].apply(lambda x: datetime.strptime(x, '%B %d, %Y').strftime('%Y-%m-%d'))

# check the data types after conversion
hospital_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   int64 
 11  chrch_affl_f                3491 non-null   int64 
 12  urban_location_f            3491 non-null   int64 
 13  children_hospital_f         3491 non-null   int6

In [196]:
hospital_df.nunique() #unique variables 

hospital_id                   3491
hospital_org_id               2377
ein                           2377
name                          3330
name_cr                       3072
street_address                3439
city                          2192
state                           51
zip_code                      3200
fips_state_and_county_code    1751
hospital_bed_count             696
chrch_affl_f                     2
urban_location_f                 2
children_hospital_f              2
memb_counc_teach_hosps_f         2
medicare_provider_number      3491
county                        1195
hospital_bed_size                3
updated_dt                       1
dtype: int64

In [197]:
hospital_df.isnull().sum() #null values 

hospital_id                   0
hospital_org_id               0
ein                           0
name                          0
name_cr                       0
street_address                0
city                          0
state                         0
zip_code                      0
fips_state_and_county_code    0
hospital_bed_count            0
chrch_affl_f                  0
urban_location_f              0
children_hospital_f           0
memb_counc_teach_hosps_f      0
medicare_provider_number      0
county                        1
hospital_bed_size             0
updated_dt                    0
dtype: int64

In [198]:
duplicate_rows = hospital_df[hospital_df.duplicated()]

print("Duplicate Rows except first occurrence:")
print(duplicate_rows)

if hospital_df.duplicated().any():
    print("Duplicates exist in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

Duplicate Rows except first occurrence:
Empty DataFrame
Columns: [hospital_id, hospital_org_id, ein, name, name_cr, street_address, city, state, zip_code, fips_state_and_county_code, hospital_bed_count, chrch_affl_f, urban_location_f, children_hospital_f, memb_counc_teach_hosps_f, medicare_provider_number, county, hospital_bed_size, updated_dt]
Index: []
No duplicates found in the DataFrame.


## Table Manipulation

In [199]:
# Hospital Information Table

# create unique ID for address
hospital_df['address_id'] = hospital_df['street_address'] + '_' + hospital_df['city'] + '_' + hospital_df['state'] + '_' + hospital_df['zip_code'] + '_' + hospital_df['fips_state_and_county_code']
hospital_df['address_id'] = hospital_df['address_id'].rank(method='dense', ascending=False).astype(int)

#create unique ID for hospital capacity
hospital_df['capacity_id'] = hospital_df['hospital_id'].astype(str) + '_' + hospital_df['hospital_bed_count'].astype(str) + '_' + hospital_df['hospital_bed_size']
hospital_df['capacity_id'] = hospital_df['capacity_id'].rank(method='dense', ascending=False).astype(int)


# create hospital information table
hospital = hospital_df[['hospital_id', 'hospital_org_id', 'name', 'ein', 'medicare_provider_number', 'address_id', 'capacity_id', 'updated_dt']]
hospital

,hospital_id,hospital_org_id,name,ein,medicare_provider_number,address_id,capacity_id,updated_dt
0,1,1,Mizell Memorial Hospital,630307951,10007,609,2381,2023-11-20
1,2,2,St Vincents East,630578923,10011,1075,1270,2023-11-20
2,3,3,Shelby Baptist Medical Center,630312913,10016,3323,667,2023-11-20
3,4,4,Callahan Eye Foundation Hosp,630459034,10018,2440,556,2023-11-20
4,5,5,Cherokee Medical Center,581973570,10022,1337,445,2023-11-20
...,...,...,...,...,...,...,...,...
3486,3487,2647,Bsw Medical Center - Austin,813040663,670136,949,730,2023-11-20
3487,3488,2304,Ascension Seton Bastrop,741109643,670143,718,729,2023-11-20
3488,3489,2648,Texas Health Hospital Frisco,831954982,670260,2901,728,2023-11-20
3489,3490,2302,Methodist Midlothian Medical Center,750800661,670300,2967,726,2023-11-20


In [200]:
# Church Affiliation Table
church_affiliation = hospital_df[['hospital_id', 'chrch_affl_f', 'updated_dt']]
church_affiliation

,hospital_id,chrch_affl_f,updated_dt
0,1,0,2023-11-20
1,2,0,2023-11-20
2,3,0,2023-11-20
3,4,0,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [201]:
# Hospital Capacity Table extracting data from hospital_df
hospital_capacity = hospital_df[['capacity_id', 'hospital_bed_size', 'hospital_bed_count', 'updated_dt']]
hospital_capacity

,capacity_id,hospital_bed_size,hospital_bed_count,updated_dt
0,2381,<100 beds,99,2023-11-20
1,1270,>299 beds,362,2023-11-20
2,667,100-299 beds,252,2023-11-20
3,556,100-299 beds,106,2023-11-20
4,445,<100 beds,60,2023-11-20
...,...,...,...,...
3486,730,<100 beds,16,2023-11-20
3487,729,<100 beds,7,2023-11-20
3488,728,<100 beds,63,2023-11-20
3489,726,<100 beds,46,2023-11-20


In [202]:
# Children Hospital Table - extracting data from hospital_df
children_hospital= hospital_df[['hospital_id', 'children_hospital_f', 'updated_dt']]
children_hospital

,hospital_id,children_hospital_f,updated_dt
0,1,0,2023-11-20
1,2,0,2023-11-20
2,3,0,2023-11-20
3,4,0,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [203]:
# Teaching Table - extracting data from hospital_df
teaching = hospital_df[['hospital_id', 'memb_counc_teach_hosps_f', 'updated_dt']]
teaching

,hospital_id,memb_counc_teach_hosps_f,updated_dt
0,1,0,2023-11-20
1,2,1,2023-11-20
2,3,0,2023-11-20
3,4,1,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [204]:
# Urban Table
urban = hospital_df[['urban_location_f', 'updated_dt','hospital_id']]
urban

,urban_location_f,updated_dt,hospital_id
0,0,2023-11-20,1
1,1,2023-11-20,2
2,1,2023-11-20,3
3,1,2023-11-20,4
4,0,2023-11-20,5
...,...,...,...
3486,1,2023-11-20,3487
3487,1,2023-11-20,3488
3488,1,2023-11-20,3489
3489,1,2023-11-20,3490


In [205]:
# Address/Location Table
Address = hospital_df[['address_id', 'street_address','city','state','zip_code','fips_state_and_county_code','county','updated_dt']]
Address

,address_id,street_address,city,state,zip_code,fips_state_and_county_code,county,updated_dt
0,609,702 Main Street,Opp,AL,36462,01039,Covington County,2023-11-20
1,1075,50 Medical Park Drive East,Birmingham,AL,35235,01073,Jefferson County,2023-11-20
2,3323,1000 First Street North,Alabaster,AL,35007,01117,Shelby County,2023-11-20
3,2440,1720 University Boulevard,Birmingham,AL,35233,01073,Jefferson County,2023-11-20
4,1337,400 Northwood Drive,Centre,AL,35960,01019,Cherokee County,2023-11-20
...,...,...,...,...,...,...,...,...
3486,949,5245 W Us 290,Austin,TX,78735,48453,Travis County,2023-11-20
3487,718,630 Highway 71 W,Bastrop,TX,78602,48021,Bastrop County,2023-11-20
3488,2901,12400 N Dallas Parkway,Frisco,TX,75033,48121,Denton County,2023-11-20
3489,2967,1201 E Highway 287,Midlothian,TX,76065,48139,Ellis County,2023-11-20


In [206]:
# connect to database

username = "jeremiahf24"
password = "$ads507password"
hostName = "ads507finalproject.mysql.database.azure.com"

conn = pymysql.connect(host=hostName,
                       port=3306,
                       user=username,
                       passwd=password, 
                       db= "ads507final", 
                       ssl={"fake_flag_to_enable_tls": True})

In [207]:
tableNames=pd.read_sql_query("""SHOW TABLES""", conn)

# print table names 
tableNames

,Tables_in_ads507final
0,address_location
1,children_hospital
2,church_affiliation
3,hospital
4,hospital_capacity
5,teaching
6,urban


In [208]:
# hospital table - convert df to list of tuples
hospital_data = [tuple(x) for x in hospital.values]

# SQL query - insert data into hospital table
sql_insert_hospital = """
INSERT INTO hospital (hospital_id, hospital_org_id, name, ein, medicare_provider_number, Address_id, capacity_id, updated_dt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# check if insertion is needed
if hospital_data:
    # execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_hospital, hospital_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error 
    finally:
        cursor.close()
else:
    print("No new data to insert into hospital table.")

Error (1062, "Duplicate entry '1' for key 'hospital.PRIMARY'")


In [209]:
# children hospital table - convert df to list of tuples
children_hospital_data = [tuple(x) for x in children_hospital.values]

# SQL query - insert data into children_hospital table
sql_insert_children_hospital = """
INSERT INTO children_hospital(hospital_id, children_hospital_f, updated_dt)
VALUES (%s, %s, %s)
"""

# check if insertion is needed
if children_hospital_data:
    # execute query
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_children_hospital, children_hospital_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into children_hospital table.")

In [210]:
# hospital_capacity table - convert df to list of tuples
hospital_capacity_data = [tuple(x) for x in hospital_capacity.values]

# SQL query - insert data into hospital_capacity table
sql_insert_hospital_capacity = """
INSERT INTO hospital_capacity (capacity_id, hospital_bed_size, hospital_bed_count, updated_dt)
VALUES (%s, %s, %s, %s)
"""

# check if insertion is needed
if hospital_capacity_data:
    # execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_hospital_capacity, hospital_capacity_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error 
    finally:
        cursor.close()
else:
    print("No new data to insert into hospital_capacity table.")

Error (1062, "Duplicate entry '2381' for key 'hospital_capacity.PRIMARY'")


In [211]:
# teaching table - convert df to list of tuples
teaching_data = [tuple(x) for x in teaching.values]

# SQL query - insert data into children_hospital table
sql_insert_teaching = """
INSERT INTO teaching (hospital_id, memb_counc_teach_hosps_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if teaching_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_teaching, teaching_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into teaching table.")

In [212]:
# church_affiliation table - convert df to list of tuples
church_affiliation_data = [tuple(x) for x in church_affiliation.values]

# SQL query - insert data into church_affiliation table
sql_insert_church_affiliation = """
INSERT INTO church_affiliation (hospital_id, chrch_affl_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if church_affiliation_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_church_affiliation, church_affiliation_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into church_affiliation table.")

In [213]:
# Address/Location table - convert df to list of tuples
address_data = [tuple(x) for x in Address.values]

# SQL query - insert data into address table
sql_insert_address_data = """
INSERT INTO address_location (Address_id, Street_address, City, State, Zip_code, fips_state_and_county_code, county, Updated_dt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# execute query 
if address_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_address_data, address_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into address_location table.")

Error (1062, "Duplicate entry '2644' for key 'address_location.PRIMARY'")


In [214]:
# Create a cursor
cursor = conn.cursor()

# Execute SQL queries to get table names
cursor.execute("SHOW TABLES")

# Fetch all the tables
tables = cursor.fetchall()

for table in tables:
        table_name = table[0]

        # Execute SQL query to fetch sample data from each table
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")

        # Fetch and print sample data
        sample_data = cursor.fetchall()
        print(f"Sample data from table '{table_name}':")
        for row in sample_data:
            print(row)

Sample data from table 'address_location':
Sample data from table 'children_hospital':
(1, 1, b'0', datetime.date(2023, 11, 20))
(2, 2, b'0', datetime.date(2023, 11, 20))
(3, 3, b'0', datetime.date(2023, 11, 20))
(4, 4, b'0', datetime.date(2023, 11, 20))
(5, 5, b'0', datetime.date(2023, 11, 20))
Sample data from table 'church_affiliation':
(1, 1, b'0', datetime.date(2023, 11, 20))
(2, 2, b'0', datetime.date(2023, 11, 20))
(3, 3, b'0', datetime.date(2023, 11, 20))
(4, 4, b'0', datetime.date(2023, 11, 20))
(5, 5, b'0', datetime.date(2023, 11, 20))
Sample data from table 'hospital':
(1, 1, 'Mizell Memorial Hospital', 630307951, 10007, 609, 2381, datetime.date(2023, 11, 20))
(2, 2, 'St Vincents East', 630578923, 10011, 1075, 1270, datetime.date(2023, 11, 20))
(3, 3, 'Shelby Baptist Medical Center', 630312913, 10016, 3323, 667, datetime.date(2023, 11, 20))
(4, 4, 'Callahan Eye Foundation Hosp', 630459034, 10018, 2440, 556, datetime.date(2023, 11, 20))
(5, 5, 'Cherokee Medical Center', 58197